## Here's an example of using Encoders on the PiStorms-GRX!
Use **Shift+Enter** to execute a cell.<br>
Or, use **Cell &rarr; Run All**.

Define some constants (also defined in GRXCom).

In [1]:
GRX_SA1_Base = 0x48
GRX_SA2_Base = 0x5E
GRX_SA3_Base = 0x74
GRX_SD1_Base = 0x8A
GRX_SD2_Base = 0xA0
_TAC2X = 5
'''
from PiStorms_GRX import GRXCom
GRX_SD1_Base = GRXCom.GRX_SD1_Base
GRX_SD2_Base = GRXCom.GRX_SD2_Base
_TAC2X = GRXCom._TAC2X
del GRXCom
'''

'\nfrom PiStorms_GRX import GRXCom\nGRX_SD1_Base = GRXCom.GRX_SD1_Base\nGRX_SD2_Base = GRXCom.GRX_SD2_Base\n_TAC2X = GRXCom._TAC2X\ndel GRXCom\n'

Create an instance of `PiStorms_GRX` to snag its `GRXCom` and thereby `mindsensors_i2c` instance.

In [2]:
from PiStorms_GRX import PiStorms_GRX
psm = PiStorms_GRX()
i2c = psm.psc.bankB

Configure the mode of BAD1 to tachometer, for it to be used as an encoder for BAS1.

In [3]:
i2c.writeArray(GRX_SD1_Base, [_TAC2X, 1])

Define some helper functions to convert a signed integer for a 4-byte array, and vice versa.

In [4]:
import struct
from functools import partial
int2arr = lambda n: map(lambda b: struct.unpack('B', b)[0], struct.pack('l', n))
arr2int = lambda n: struct.unpack('l', ''.join(map(partial(struct.pack, 'B'), n)))[0]

Set the target of BAD1 (associated with BAS1) to encoder position 100.<br>
Try changing 100 to a different value and use **Shift+Enter** to run it!

In [35]:
i2c.writeArray(GRX_SD1_Base+8, int2arr(-40))

Read the current encoder value on BAD1

In [36]:
arr2int(i2c.readArray(GRX_SD1_Base+4, 4))

-41

Monitor the encoder value

In [28]:
from __future__ import print_function
import time
getenc = lambda: arr2int(i2c.readArray(GRX_SD1_Base+4, 4))
def p(n):
    v = int(str(abs(n))[:2]) # most-significant two digits
    c = (31 if n<1 else 32) + (abs(n)>100)*60 # ANSI color code
    print(u"\r{num:5d}: \033[{color}m{bar}{blank}\033[0m".format(
            num=n, bar=unichr(0x2588)*v, blank=" "*(100-v), color=c), end="")
while not psm.isKeyPressed():
    p(getenc())
    time.sleep(0.1)

    0:                                                                                                     

KeyboardInterrupt: 